In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [4]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [8]:
prompt = """
<human>: Find both the full acronym with its expansion and, if unavailable, the acronym itself in the following sentence, The recognition should be case-insensitive: "What is here called controlled natural language( CNL) has traditionally been given many different names. In this paper, we introduce CoQA, a Conversational Question Answering dataset for measuring the ability of machines to participate in a question- answering style conversation. in: Proceedings of SODA 98, Ninth Annual ACM- SIAM Symposium on Discrete Algorithms, San Francisco, CA, USA, 1998, pp. But is the existing HPC I / O stack still suitable for new NVM devices like Intel Optane? For a fixed and known, we apply a successive convex approximation( SCA) approach to transform the non- convex problem into a sequence of relaxed convex subproblems. Section II introduces issues relevant to VMP. A near optimal solution is proposed using Geometric Programming( GP) to optimally solve the problem in a time- slotted period. b), this regularization induces an output distribution that is close to an input distribution in CT values, and aims to ensure that the generators translate the sparse artifacts rather than dense anatomical structures. The Principal Component Analysis( PCA) is used for dimension reduction. This should certainly be true of the New Technology File System( NTFS), since NTFS is currently the most common file system in use."
<assistant>:
""".strip()

In [9]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [10]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Find both the full acronym with its expansion and, if unavailable, the acronym itself in the following sentence, The recognition should be case-insensitive: "What is here called controlled natural language( CNL) has traditionally been given many different names. In this paper, we introduce CoQA, a Conversational Question Answering dataset for measuring the ability of machines to participate in a question- answering style conversation. in: Proceedings of SODA 98, Ninth Annual ACM- SIAM Symposium on Discrete Algorithms, San Francisco, CA, USA, 1998, pp. But is the existing HPC I / O stack still suitable for new NVM devices like Intel Optane? For a fixed and known, we apply a successive convex approximation( SCA) approach to transform the non- convex problem into a sequence of relaxed convex subproblems. Section II introduces issues relevant to VMP. A near optimal solution is proposed using Geometric Programming( GP) to optimally solve the problem in a time- slotted period. b), t

# Prep dataset

In [ ]:
data = load_dataset("csv", data_files="/content/falcon_train.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Text', 'Acronyms'],
        num_rows: 1257
    })
})

In [ ]:
data["train"][0]

{'Text': 'Find both the full acronym with its expansion and, if unavailable, the acronym itself in the following sentence, The recognition should be case-insensitive: "What is here called controlled natural language( CNL) has traditionally been given many different names. In this paper, we introduce CoQA, a Conversational Question Answering dataset for measuring the ability of machines to participate in a question- answering style conversation. in: Proceedings of SODA 98, Ninth Annual ACM- SIAM Symposium on Discrete Algorithms, San Francisco, CA, USA, 1998, pp. But is the existing HPC I / O stack still suitable for new NVM devices like Intel Optane? For a fixed and known, we apply a successive convex approximation( SCA) approach to transform the non- convex problem into a sequence of relaxed convex subproblems. Section II introduces issues relevant to VMP. A near optimal solution is proposed using Geometric Programming( GP) to optimally solve the problem in a time- slotted period. b), 

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Text"]}
<assistant>: {data_point["Acronyms"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/1257 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['Text', 'Acronyms', 'input_ids', 'attention_mask'],
    num_rows: 1257
})

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.095800
2,3.082400
3,3.192300
4,3.151600
5,3.030100
6,3.197900
7,3.013200
8,2.954800
9,2.781700
10,2.876600


TrainOutput(global_step=314, training_loss=2.3556277212823273, metrics={'train_runtime': 3975.3911, 'train_samples_per_second': 0.316, 'train_steps_per_second': 0.079, 'total_flos': 1.3467778955644416e+16, 'train_loss': 2.3556277212823273, 'epoch': 1.0})

# Save trained model

In [ ]:
model.save_pretrained("/content/drive/MyDrive/LLM/falcon/falcon_final")

In [11]:
PEFT_MODEL = "/content/drive/MyDrive/LLM/falcon/falcon_final"


In [12]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run the finetuned model

In [13]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [14]:
%%time
device = "cuda:0"

prompt = """
<human>: Find both the full acronym with its expansion and, if unavailable, the acronym itself in the following sentence, The recognition should be case-insensitive: "What is here called controlled natural language( CNL) has traditionally been given many different names. In this paper, we introduce CoQA, a Conversational Question Answering dataset for measuring the ability of machines to participate in a question- answering style conversation. in: Proceedings of SODA 98, Ninth Annual ACM- SIAM Symposium on Discrete Algorithms, San Francisco, CA, USA, 1998, pp. But is the existing HPC I / O stack still suitable for new NVM devices like Intel Optane? For a fixed and known, we apply a successive convex approximation( SCA) approach to transform the non- convex problem into a sequence of relaxed convex subproblems. Section II introduces issues relevant to VMP. A near optimal solution is proposed using Geometric Programming( GP) to optimally solve the problem in a time- slotted period. b), this regularization induces an output distribution that is close to an input distribution in CT values, and aims to ensure that the generators translate the sparse artifacts rather than dense anatomical structures. The Principal Component Analysis( PCA) is used for dimension reduction. This should certainly be true of the New Technology File System( NTFS), since NTFS is currently the most common file system in use."
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Find both the full acronym with its expansion and, if unavailable, the acronym itself in the following sentence, The recognition should be case-insensitive: "What is here called controlled natural language( CNL) has traditionally been given many different names. In this paper, we introduce CoQA, a Conversational Question Answering dataset for measuring the ability of machines to participate in a question- answering style conversation. in: Proceedings of SODA 98, Ninth Annual ACM- SIAM Symposium on Discrete Algorithms, San Francisco, CA, USA, 1998, pp. But is the existing HPC I / O stack still suitable for new NVM devices like Intel Optane? For a fixed and known, we apply a successive convex approximation( SCA) approach to transform the non- convex problem into a sequence of relaxed convex subproblems. Section II introduces issues relevant to VMP. A near optimal solution is proposed using Geometric Programming( GP) to optimally solve the problem in a time- slotted period. b), t

Index of the generated text

In [15]:
import pandas as pd

# Ask the user for input text
user_input = input("Please enter your text: ")

# Process the input text
processed_text = f"Find both the full acronym with its expansion and, if unavailable, the acronym itself in the following sentence. The recognition should be case-insensitive: \"{user_input}\""

# Create a data frame named 'df'
df = pd.DataFrame({'Text': [processed_text]})

# Display the data frame
print(df)


Please enter your text: KDD, New York, NY, USA, 2002, pp. Similarly, it is possible to construct a blacklist formed by criminals, or by countries that are considered to be non- cooperative against money laundering, as is the case of the Financial Action Task Force( FATF) blacklist. The problem is formulated as an integer linear programming( ILP) problem whose objective is to jointly minimize the inter- cell interference and the fronthaul bandwidth utilization. Within the domain of discrete and symbolic process modelling, the present work aims at providing a framework that may be useful to validate and further analyse models formulated in very general classes of languages, the most important being the-calculus( comprising as a subset Linear Temporal Logic( LTL) and Computation Tree Logic( CTL)) on the syntactic, and several types of universal coalgebras on the semantic level. Furthermore, we say that a nonempty set is a strongly connected component( abbreviated SCC) of if it is strongly

In [16]:
import pandas as pd


# Define a function to process the text and get the model's output
def get_model_output(text):
    prompt = f"<human>: {text}\n<assistant>:"
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

last_row = df.iloc[-1]

model_output = get_model_output(last_row['Text'])

# Create a new DataFrame with the existing column and the model's output
selected_rows = pd.DataFrame({
    'Text': [last_row['Text']],
    'Output': [model_output]
})

# Print the new DataFrame
print(selected_rows)

                                                Text  \
0  Find both the full acronym with its expansion ...   

                                              Output  
0  <human>: Find both the full acronym with its e...  


In [17]:
import re

def extract_acronyms(text):
    # Find the content after <assistant> and before the first closing ]
    match = re.search(r'<assistant>:\s*\[(.*?)\]', text)
    if match:
        # Split the content by comma
        items = [item.strip() for item in match.group(1).split(',')]
        return '[' + ', '.join(items) + ']'
    else:
        # If no closing bracket is found, extract everything after the opening bracket
        match = re.search(r'<assistant>:\s*\[(.*)', text)
        if match:
            items = [item.strip() for item in match.group(1).split(',')]
            return '[' + ', '.join(items) + ']'
    return []

# Applying the function to the 'Model Output' column
selected_rows['Changed_output'] = selected_rows['Output'].apply(extract_acronyms)

# Creating a new DataFrame with 'Combined Acronyms' and 'Assistant Output' columns
new_df = selected_rows[['Text', 'Output', 'Changed_output']]

import pandas as pd
import ast

def split_list(input_list):
    output_list = []
    for item in input_list:
        if ':' in item:
            key, value = item.split(':', 1)  # Split at the first colon
            output_list.append(key.strip())  # Append the key (abbreviation) to the output list
            output_list.append(value.strip())  # Append the value (full form) to the output list
        else:
            output_list.append(item)  # If no colon, just append the item as it is
    return output_list

def safe_eval(s):
    try:
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        # Return the original string if there's an error
        return s

new_df['acronym'] = new_df['Changed_output'].apply(lambda x: safe_eval(x) if isinstance(x, str) else x)

# Apply the function to each row of the DataFrame
new_df['acronym'] = new_df['acronym'].apply(split_list)

print(new_df)

                                                Text  \
0  Find both the full acronym with its expansion ...   

                                              Output  \
0  <human>: Find both the full acronym with its e...   

                                      Changed_output  \
0  ['KDD', 'NY', 'USA', 'FATF: Financial Action T...   

                                             acronym  
0  [KDD, NY, USA, FATF, Financial Action Task For...  


In [18]:
import re

def find_indices(text, search_list):
    # Extract the content within the double quotes
    start_quote = text.find('"')
    end_quote = text.rfind('"')
    content = text[start_quote+1:end_quote]

    indices = []
    for item in search_list:
        start_index = content.lower().find(item.lower())
        if start_index != -1:
            end_index = start_index + len(item) - 1
            indices.append((item, start_index, end_index))
    return indices

# Assuming you have a dataframe called new_df
text = new_df['Text'].iloc[0]  # This will get the first row's value as a string

# Convert the Series to a string and then extract terms using regex
terms_string = str(new_df['acronym'].iloc[0])
terms = re.findall(r"'(.*?)'", terms_string)

results = find_indices(text, terms)
for item, start, end in results:
    print(f"'{item}' index ({start}, {end})")
print(terms)

'KDD' index (0, 2)
'NY' index (15, 16)
'USA' index (19, 21)
'FATF' index (242, 245)
'Financial Action Task Force' index (213, 239)
'ILP' index (319, 321)
'integer linear programming' index (291, 316)
'LTL' index (742, 744)
'Linear Temporal Logic' index (719, 739)
'CTL' index (775, 777)
'Computation Tree Logic' index (751, 772)
'SCC' index (951, 953)
'strongly connected component' index (909, 936)
'EL' index (383, 384)
'Ensemble Learning' index (1076, 1092)
'LSB' index (1144, 1146)
'Least Squares Boosting' index (1120, 1141)
'SW' index (1299, 1300)
'sliding window' index (1339, 1352)
'LSW' index (1298, 1300)
'SW' index (1299, 1300)
'sliding window' index (1339, 1352)
'LSW' index (1298, 1300)
'SW' index (1299, 1300)
'SGBR' index (1545, 1548)
'RIMD' index (1554, 1557)
'BM' index (1737, 1738)
'bare metal' index (1725, 1734)
'VM' index (1761, 1762)
'virtual machine' index (1744, 1758)
'CR' index (95, 96)
'cascaded regression' index (1804, 1822)
['KDD', 'NY', 'USA', 'FATF', 'Financial Action

Acuraccy of the model

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/falcon_test.csv')

# Define the range of rows you want to process
start_row = 0 # Change this to the starting row index (0-based)
end_row = 314   # Change this to the ending row index (exclusive)

# Select the desired rows
selected_rows = df.iloc[start_row:end_row]

# Define a function to process the text and get the model's output
def get_model_output(text):
    prompt = f"<human>: {text}\n<assistant>:"
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply the function to the selected rows and store the results in a new column
selected_rows['Output1'] = selected_rows['Text'].apply(get_model_output)

# Print the selected rows with the model's output and the comparison result
print(selected_rows[['Text', 'Acronyms', 'Output1']])


In [ ]:

import re

def extract_acronyms(text):
    # Find the content after <assistant> and before the first closing ]
    match = re.search(r'<assistant>:\s*\[(.*?)\]', text)
    if match:
        # Split the content by comma
        items = [item.strip() for item in match.group(1).split(',')]
        return '[' + ', '.join(items) + ']'
    else:
        # If no closing bracket is found, extract everything after the opening bracket
        match = re.search(r'<assistant>:\s*\[(.*)', text)
        if match:
            items = [item.strip() for item in match.group(1).split(',')]
            return '[' + ', '.join(items) + ']'
    return []

# Applying the function to the 'Model Output' column
selected_rows['Output'] = selected_rows['Output1'].apply(extract_acronyms)

# Creating a new DataFrame with 'Combined Acronyms' and 'Assistant Output' columns
new_df = selected_rows[['Acronyms', 'Output1','Output']]

new_df = new_df.assign(**{'Text': selected_rows['Text']})

print(new_df)

In [ ]:
selected_rows

In [ ]:
import pandas as pd
import ast

def split_list(input_list):
    output_list = []
    for item in input_list:
        if ':' in item:
            key, value = item.split(':', 1)  # Split at the first colon
            output_list.append(key.strip())  # Append the key (abbreviation) to the output list
            output_list.append(value.strip())  # Append the value (full form) to the output list
        else:
            output_list.append(item)  # If no colon, just append the item as it is
    return output_list

def safe_eval(s):
    try:
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        # Return None if there's an error
        return None

new_df['acronym'] = new_df['Acronyms'].apply(lambda x: safe_eval(x) if isinstance(x, str) else x)
new_df['out'] = new_df['Output'].apply(lambda x: safe_eval(x) if isinstance(x, str) else x)

# Filter out rows where either 'acronym' or 'out' is None (i.e., encountered an error)
filtered_df = new_df.dropna(subset=['acronym', 'out'])

# Apply the function to each row of the filtered DataFrame
filtered_df['acronym'] = filtered_df['acronym'].apply(split_list)
filtered_df['out'] = filtered_df['out'].apply(split_list)

print(filtered_df)


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

def calculate_metrics(df, actual_column, predicted_column):
    confusion_matrix_array = [[0, 0], [0, 0]]  # Assuming binary classification

    for row in df.itertuples(index=False):
        actual_labels = getattr(row, actual_column)
        predicted_labels = getattr(row, predicted_column)
        for actual_label in actual_labels:
            if actual_label in predicted_labels:
                # True Positive
                confusion_matrix_array[0][0] += 1
            else:
                # False Negative
                confusion_matrix_array[0][1] += 1

        for predicted_label in predicted_labels:
            if predicted_label not in actual_labels:
                # False Positive
                confusion_matrix_array[1][0] += 1

    confusion_matrix_df = pd.DataFrame(confusion_matrix_array, index=['Actual', 'Not Actual'], columns=['Predicted', 'Not Predicted'])

    true_positive = confusion_matrix_df.at['Actual', 'Predicted']
    false_positive = confusion_matrix_df.at['Not Actual', 'Predicted']
    false_negative = confusion_matrix_df.at['Actual', 'Not Predicted']

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    f1_score = 2 * (precision * recall) / (precision + recall)
    accuracy = (true_positive + confusion_matrix_df.at['Not Actual', 'Not Predicted']) / confusion_matrix_df.sum().sum()

    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'accuracy': accuracy,
        'confusion_matrix': confusion_matrix_df
    }

    return metrics

# Specify the actual and predicted column names
actual_column_name = 'acronym'
predicted_column_name = 'out'

# Calculate metrics using the function
metrics = calculate_metrics(filtered_df, actual_column_name, predicted_column_name)

# Display metrics
print("Confusion Matrix:")
print(metrics['confusion_matrix'])
print("\nMetrics:")
print(f"Precision: {metrics['precision']:.2f}")
print(f"Recall: {metrics['recall']:.2f}")
print(f"F1-Score: {metrics['f1_score']:.2f}")
print(f"Accuracy: {metrics['accuracy']:.2f}")


In [ ]:
from google.colab import files

# Save the file to the virtual machine
selected_rows.to_csv('filename.csv')

# Download the file to your local machine
files.download('filename.csv')


In [ ]:
from google.colab import files

# Save the file to the virtual machine
filtered_df.to_csv('filename.csv')

# Download the file to your local machine
files.download('filename.csv')
